# This notebook will aim to use the trained models weights and load it on the inference model to generate predictions and novel sequences

In [11]:
from models.arcs import generate, GenPhiloTextA
from utilities.loaders import load_file
from utilities.preprocessors import preprocess, get_chars, map_value_to_index, decode_predictions
import tensorflow as tf

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load raw data

In [12]:
corpus = load_file('./data/notes.txt')
corpus = preprocess(corpus)
chars = get_chars(corpus)
char_to_idx = map_value_to_index(chars)
idx_to_char = map_value_to_index(chars, inverted=True)

In [13]:
char_to_idx.get_vocabulary()

['[UNK]',
 '\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '´',
 'ç',
 'é',
 'ï',
 '—',
 '…']

In [14]:
n_unique = len(char_to_idx.get_vocabulary())

# Declare same hyper params used in training model

In [15]:
emb_dim = 256
n_a = 512
T_x = 100
dense_layers_dims = [n_unique]
batch_size = 128
alpha = 1e-3
lambda_ = 0.8
drop_prob = 0.4
normalize = False
n_epochs =  100

# Declare sample input to build inference model and access .summary()

In [16]:
sample_input = tf.random.uniform(shape=(1, T_x), minval=0, maxval=n_unique - 1, dtype=tf.int32)
sample_h = tf.zeros(shape=(1, n_a))
sample_c = tf.zeros(shape=(1, n_a))

# Redeclare architecture by passing the same hyper params used in training and then load weights

In [17]:
saved_model = GenPhiloTextA(emb_dim=emb_dim, n_a=n_a, n_unique=n_unique, dense_layers_dims=dense_layers_dims, drop_prob=drop_prob, normalize=normalize)
saved_model(sample_input)

<tf.Tensor: shape=(1, 100, 57), dtype=float32, numpy=
array([[[-0.00540989, -0.00015633,  0.00198572, ..., -0.00101178,
         -0.00120615, -0.00105561],
        [-0.00121182,  0.00344092,  0.00260131, ..., -0.00104574,
          0.00100229,  0.00843915],
        [ 0.00267117,  0.01026063,  0.00286845, ...,  0.0010662 ,
          0.00959603,  0.00309708],
        ...,
        [-0.0080949 ,  0.0046614 , -0.00214528, ..., -0.00063614,
         -0.00011534, -0.00825164],
        [-0.010578  ,  0.00026269, -0.00389015, ..., -0.00127934,
          0.00208991, -0.00692616],
        [ 0.00742032, -0.00104589, -0.00278196, ..., -0.00734997,
         -0.00323965, -0.00666001]]], dtype=float32)>

In [24]:
sample_input = tf.random.uniform(shape=(1, T_x + 10), minval=0, maxval=n_unique - 1, dtype=tf.int32)
saved_model(sample_input)

<tf.Tensor: shape=(1, 110, 57), dtype=float32, numpy=
array([[[-0.99894917, -0.05772941,  0.7268927 , ..., -0.9724182 ,
         -0.9088973 , -0.94427675],
        [-1.1066761 , -0.06039706,  1.2362472 , ..., -1.0789745 ,
         -0.9914653 , -1.0287079 ],
        [-1.3930281 , -0.06760611,  2.5738266 , ..., -1.3620644 ,
         -1.2109687 , -1.2518831 ],
        ...,
        [-1.5018518 , -0.07016251,  3.0781136 , ..., -1.4694932 ,
         -1.2944503 , -1.3362739 ],
        [-1.5018518 , -0.07016251,  3.0781136 , ..., -1.4694932 ,
         -1.2944503 , -1.3362739 ],
        [-1.5018518 , -0.07016251,  3.0781136 , ..., -1.4694932 ,
         -1.2944503 , -1.3362739 ]]], dtype=float32)>

In [18]:
saved_model.trainable_weights

[<tf.Variable 'gen_philo_text_a_1/character-lookup/embeddings:0' shape=(57, 256) dtype=float32, numpy=
 array([[ 0.02210325,  0.03110288, -0.037651  , ...,  0.04402642,
          0.02510992,  0.03947211],
        [ 0.04615412,  0.01848548, -0.00974838, ...,  0.02000702,
         -0.00067936, -0.01324873],
        [-0.02544125, -0.02470543,  0.02690467, ..., -0.03230782,
          0.02221466, -0.03676935],
        ...,
        [-0.01678227, -0.01716178,  0.02083695, ..., -0.02941795,
          0.00815401, -0.0451331 ],
        [ 0.02076752,  0.00204693, -0.04578961, ..., -0.00936799,
         -0.03868567,  0.02656769],
        [ 0.0333526 ,  0.0111682 , -0.04376334, ...,  0.00224353,
          0.01670386,  0.02239729]], dtype=float32)>,
 <tf.Variable 'gen_philo_text_a_1/lstm-layer/lstm_cell/kernel:0' shape=(256, 2048) dtype=float32, numpy=
 array([[ 0.01321833,  0.0041265 ,  0.04305574, ..., -0.02544646,
          0.05080582,  0.03523806],
        [ 0.04631002,  0.00780468, -0.02287318,

In [19]:
saved_model.summary()

Model: "gen_philo_text_a_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 character-lookup (Embeddin  multiple                  14592     
 g)                                                              
                                                                 
 lstm-layer (LSTM)           multiple                  1574912   
                                                                 
 dense-layer-0 (Dense)       multiple                  29241     
                                                                 
Total params: 1618745 (6.18 MB)
Trainable params: 1618745 (6.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
saved_model.load_weights(filepath='./saved/weights/notes_gen_philo_text_a_100_3.0300.h5')

In [22]:
saved_model.trainable_weights

[<tf.Variable 'gen_philo_text_a_1/character-lookup/embeddings:0' shape=(57, 256) dtype=float32, numpy=
 array([[ 1.5778426e-33,  8.5969997e-32,  9.0914019e-33, ...,
          2.6084619e-31,  1.2379388e-31,  6.8241091e-32],
        [-1.7392811e-06,  3.5470998e-06, -1.1924452e-06, ...,
         -1.8112476e-06,  1.5420478e-06, -1.3949349e-06],
        [-2.6034151e-05,  2.2033426e-05, -1.9344105e-05, ...,
         -2.8262819e-05,  2.7131497e-05, -2.2557864e-05],
        ...,
        [-8.5676251e-11,  1.9831338e-11,  8.3653984e-10, ...,
         -1.2872679e-10,  1.4269575e-11,  2.9089064e-10],
        [-5.6282662e-08, -9.0218379e-08, -1.5125519e-08, ...,
          1.4084819e-07, -4.6915076e-08, -5.9459062e-08],
        [ 5.1603475e-08, -8.0799829e-09, -5.9060262e-08, ...,
         -8.0631750e-08, -6.6796417e-08, -9.6390124e-08]], dtype=float32)>,
 <tf.Variable 'gen_philo_text_a_1/lstm-layer/lstm_cell/kernel:0' shape=(256, 2048) dtype=float32, numpy=
 array([[ 0.0263859 , -0.03103871,  0.011

# Extract trainable layers

In [21]:
# decode_predictions(pred_ids, idx_to_char)